In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns',1000000000000)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
data.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0


In [4]:
data.set_index('Id',inplace=True)
test.set_index('Id',inplace=True)

In [5]:
data.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0
1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0
2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0
3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0
4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0


In [6]:
test.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0
2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0
3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0
4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0


In [7]:
data['Open Date'] = data['Open Date'].astype('datetime64')

In [8]:
test['Open Date'] = test['Open Date'].astype('datetime64')

In [9]:
data['Type'].value_counts()

FC    76
IL    60
DT     1
Name: Type, dtype: int64

In [10]:
test['Type'].value_counts()

FC    57019
IL    40447
DT     2244
MB      290
Name: Type, dtype: int64

In [11]:
test['Type'] = test['Type'].replace({'MB':'DT'})

In [12]:
test['Type'].value_counts()

FC    57019
IL    40447
DT     2534
Name: Type, dtype: int64

In [13]:
label_encoder = LabelEncoder()

In [14]:
data['City Group'] = label_encoder.fit_transform(data['City Group'])

In [15]:
test['City Group'] = label_encoder.transform(test['City Group'])

In [16]:
one_hot_encode = OneHotEncoder(drop='first',sparse=False)

In [17]:
a = one_hot_encode.fit_transform(np.array(data['Type']).reshape(-1,1))

In [18]:
b = one_hot_encode.transform(np.array(test['Type']).reshape(-1,1))

In [19]:
data = data.join(pd.DataFrame(a,columns=list(one_hot_encode.get_feature_names(['Type']))))

In [20]:
data.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue,Type_FC,Type_IL
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1999-07-17,İstanbul,0,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0,0.0,1.0
1,2008-02-14,Ankara,0,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0,1.0,0.0
2,2013-03-09,Diyarbakır,1,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0,0.0,1.0
3,2012-02-02,Tokat,1,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0,0.0,1.0
4,2009-05-09,Gaziantep,1,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0,0.0,1.0


In [21]:
test = test.join(pd.DataFrame(b,columns=list(one_hot_encode.get_feature_names(['Type']))))

In [22]:
test.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,Type_FC,Type_IL
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-01-22,Niğde,1,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0
1,2011-03-18,Konya,1,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,0.0,1.0
2,2013-10-30,Ankara,0,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0
3,2013-05-06,Kocaeli,1,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0,0.0,1.0
4,2013-07-31,Afyonkarahisar,1,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0


In [23]:
data['year'] = data['Open Date'].dt.year
data['month'] = data['Open Date'].dt.month
data['day'] = data['Open Date'].dt.day

In [24]:
data.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1999-07-17,İstanbul,0,IL,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0,0.0,1.0,1999,7,17
1,2008-02-14,Ankara,0,FC,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0,1.0,0.0,2008,2,14
2,2013-03-09,Diyarbakır,1,IL,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0,0.0,1.0,2013,3,9
3,2012-02-02,Tokat,1,IL,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0,0.0,1.0,2012,2,2
4,2009-05-09,Gaziantep,1,IL,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0,0.0,1.0,2009,5,9


In [25]:
test['year'] = test['Open Date'].dt.year
test['month'] = test['Open Date'].dt.month
test['day'] = test['Open Date'].dt.day

In [26]:
test.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2011-01-22,Niğde,1,FC,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0,2011,1,22
1,2011-03-18,Konya,1,IL,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,0.0,1.0,2011,3,18
2,2013-10-30,Ankara,0,FC,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0,2013,10,30
3,2013-05-06,Kocaeli,1,IL,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0,0.0,1.0,2013,5,6
4,2013-07-31,Afyonkarahisar,1,FC,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0,2013,7,31


In [27]:
data.drop(['Open Date','City','Type'],1,inplace=True)

In [28]:
data.head()

,City Group,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,4,5.0,4.0,4.0,2,2,5,4,5,5,3,5,5.0,1,2,2,2,4,5,4,1,3,3,1,1,1.0,4.0,2.0,3.0,5,3,4,5,5,4,3,4,5653753.0,0.0,1.0,1999,7,17
1,0,4,5.0,4.0,4.0,1,2,5,5,5,5,1,5,5.0,0,0,0,0,0,3,2,1,3,2,0,0,0.0,0.0,3.0,3.0,0,0,0,0,0,0,0,0,6923131.0,1.0,0.0,2008,2,14
2,1,2,4.0,2.0,5.0,2,3,5,5,5,5,2,5,5.0,0,0,0,0,0,1,1,1,1,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,2055379.0,0.0,1.0,2013,3,9
3,1,6,4.5,6.0,6.0,4,4,10,8,10,10,8,10,7.5,6,4,9,3,12,20,12,6,1,10,2,2,2.5,2.5,2.5,7.5,25,12,10,6,18,12,12,6,2675511.0,0.0,1.0,2012,2,2
4,1,3,4.0,3.0,4.0,2,2,5,5,5,5,2,5,5.0,2,1,2,1,4,2,2,1,2,1,2,3,3.0,5.0,1.0,3.0,5,1,3,2,3,4,3,3,4316715.0,0.0,1.0,2009,5,9


In [29]:
test.drop(['Open Date','City','Type'],1,inplace=True)

In [30]:
test.head()

,City Group,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37,Type_FC,Type_IL,year,month,day
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,4.0,4.0,4.0,1,2,5,4,5,5,5,3,4.0,0,0,0,2,0,5,5,3,1,4,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0,2011,1,22
1,1,3,4.0,4.0,4.0,2,2,5,3,4,4,2,4,5.0,0,0,0,0,0,5,5,3,2,1,0,0,0.0,0.0,1.0,3.0,0,0,0,0,0,0,0,0,0.0,1.0,2011,3,18
2,0,3,4.0,4.0,4.0,2,2,5,4,4,5,4,5,5.0,0,0,0,0,0,5,5,5,5,5,0,0,0.0,0.0,2.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0,2013,10,30
3,1,2,4.0,4.0,4.0,2,3,5,4,5,4,3,4,5.0,0,0,0,0,4,4,4,3,2,2,0,0,0.0,0.0,2.0,3.0,0,4,0,0,0,0,0,0,0.0,1.0,2013,5,6
4,1,2,4.0,4.0,4.0,1,2,5,4,5,4,3,5,4.0,0,0,0,0,0,1,5,3,1,1,0,0,0.0,0.0,5.0,3.0,0,0,0,0,0,0,0,0,1.0,0.0,2013,7,31


In [31]:
data.shape, test.shape

((137, 44), (100000, 43))

In [32]:
(30*43)/100

12.9

In [33]:
X = data.drop('revenue',1)
y = data['revenue']

In [34]:
kfold = KFold()

params = {'n_estimators':130,
         'learning_rate':0.04,
          'max_depth':1,
          'subsample':0.9,
          'loss':'huber',
          'criterion':'mae',
         'random_state':500}

In [35]:
score = 0
gbm_results = pd.DataFrame()
for i,(train_index,val_index) in enumerate(kfold.split(X,y)):
    X_train,y_train = X.iloc[train_index],y.iloc[train_index]
    X_test,y_test = X.iloc[val_index],y.iloc[val_index]
    
    model = GradientBoostingRegressor(**params)
    print(f'************ fold {i}**************')
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    print(rmse)
    score += rmse/5
    result = model.predict(test)
    
    gbm_results['Fold'+str(i)] = result
    
print(f'Average score is {score}')

************ fold 0**************
3416494.060995665
************ fold 1**************
1652870.1611265917
************ fold 2**************
2668507.115666048
************ fold 3**************
2449075.884909259
************ fold 4**************
2065996.696286855
Average score is 2450588.783796884


In [36]:
gbm_results

,Fold0,Fold1,Fold2,Fold3,Fold4
0,4.745699e+06,4.358405e+06,4.256680e+06,4.297009e+06,4.364255e+06
1,3.253553e+06,2.709886e+06,3.331491e+06,3.847293e+06,3.407796e+06
2,3.544594e+06,3.047827e+06,4.635756e+06,2.772256e+06,3.247895e+06
3,3.551029e+06,3.005025e+06,3.500815e+06,2.755893e+06,3.200778e+06
4,3.762002e+06,2.958353e+06,3.863413e+06,2.824220e+06,3.256778e+06
...,...,...,...,...,...
99995,4.469514e+06,5.494474e+06,4.570800e+06,4.503052e+06,4.743847e+06
99996,3.147938e+06,3.308717e+06,3.378043e+06,3.809951e+06,3.381504e+06
99997,3.874982e+06,3.317621e+06,4.818013e+06,4.076534e+06,3.959173e+06
99998,3.620098e+06,3.268894e+06,3.971390e+06,3.195792e+06,3.521806e+06


In [37]:
kfold = KFold(n_splits=7)

params = {'max_features':0.09,
          'n_estimators':125,
          'max_depth':6,
          'min_samples_leaf':5,
         'random_state':500}

In [38]:
score = 0
rf_results = pd.DataFrame()
for i,(train_index,val_index) in enumerate(kfold.split(X,y)):
    X_train,y_train = X.iloc[train_index],y.iloc[train_index]
    X_test,y_test = X.iloc[val_index],y.iloc[val_index]
    
    model = RandomForestRegressor(**params)
    print(f'************ fold {i}**************')
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,preds))
    print(rmse)
    score += rmse/5
    result = model.predict(test)
    
    rf_results['Fold'+str(i)] = result
    
print(f'Average score is {score}')

************ fold 0**************
3691202.9138741638
************ fold 1**************
1849585.3011340688
************ fold 2**************
1827807.194944989
************ fold 3**************
2939295.161650287
************ fold 4**************
1400584.9839847945
************ fold 5**************
2719605.7532265154
************ fold 6**************
1940364.892485556
Average score is 3273689.240260075


In [39]:
rf_results

,Fold0,Fold1,Fold2,Fold3,Fold4,Fold5,Fold6
0,4.425907e+06,4.486321e+06,4.276302e+06,4.135180e+06,4.374900e+06,4.168328e+06,4.332403e+06
1,3.913110e+06,4.070180e+06,4.130858e+06,4.094312e+06,4.252182e+06,4.031792e+06,3.982240e+06
2,4.303183e+06,4.722614e+06,4.494560e+06,4.500149e+06,4.613596e+06,4.291447e+06,4.486566e+06
3,3.765363e+06,3.987716e+06,3.973599e+06,3.916338e+06,3.930257e+06,3.948701e+06,3.809671e+06
4,4.384226e+06,4.655775e+06,4.553169e+06,4.569580e+06,4.414564e+06,4.347740e+06,4.356081e+06
...,...,...,...,...,...,...,...
99995,4.385191e+06,5.173163e+06,4.838920e+06,4.679938e+06,4.802391e+06,4.894673e+06,4.660007e+06
99996,3.208054e+06,3.275143e+06,3.259895e+06,3.366496e+06,3.413740e+06,3.215213e+06,3.233043e+06
99997,4.790323e+06,4.985698e+06,4.422143e+06,4.895675e+06,5.108373e+06,4.713240e+06,4.947466e+06
99998,4.705220e+06,4.773628e+06,4.386620e+06,4.672743e+06,4.550317e+06,4.617145e+06,4.596787e+06


In [40]:
sub = pd.read_csv('../data/sampleSubmission.csv')

In [42]:
sub['Prediction'] = np.mean(gbm_results,1)*0.58+ np.mean(rf_results,1)*0.42

In [43]:
sub.to_csv('../data/blendsub.csv',index=False)

In [44]:
sub = pd.read_csv('../data/sampleSubmission.csv')

In [45]:
ilori = pd.read_csv('../data/submission_xgb_000.csv')

In [52]:
sub['Prediction'] = ilori['Prediction']*0.95 + np.mean(gbm_results,1)*0.05

In [53]:
sub.to_csv('../data/iloriblend.csv',index=False)